## CTA simulations

First we import the libraries.

In [14]:
# import the required libraries
import os
import ctools
import numpy as np

We can make a folder to store the example data that will be created in this tutorial and used in the futher ones. 

In [15]:
if not os.path.isdir('../data'):
    os.mkdir('../data')

For simplicity let's make a wrapper of the ctools MC simulator. We can write a function with some default parameters in order to invoke it when needed.

In [25]:
#creazione simulazione
def run_ctobssim(model, pointing, output, energy=(0.03, 150), time=(0, 1200), fov=5, caldb='prod3b-v2', irf='South_z40_0.5h', seed=1):
    sim = ctools.ctobssim()
    sim["inmodel"] = model #inserisce un modello pre-esistente all'interno del tool di generazione mappe
    sim["outevents"] = output #estrae una mappa creata in estensione .fits
    sim["caldb"] = caldb #calibration database, inserito nella fase di configurazione come zip
    sim["irf"] = irf #Instrumental response function, mappa tra flusso di fotoni in ingresso e gli eventi detected
    sim["ra"] = pointing[0] #Right Ascension of CTA pointing (dove punta nello spazio il telescopio)
    sim["dec"] = pointing[1] #Declination of CTA pointing (dove punta nello spazio il telescopio)
    sim["rad"] = fov #field of view, simulation cone radius, 5 rad mi pare un botto
    sim["tmin"] = time[0] #start time
    sim["tmax"] = time[1] #stop time of the exposure
    sim["emin"] = energy[0] #Lower energy limit of simulated events (in TeV).
    sim["emax"] = energy[1] #Upper energy limit of simulated events (in TeV).
    sim["seed"] = seed #random number generator for the simulations with a seed value provided by the hidden seed parameter
    sim.execute()
    # if you want to run in memory:
    #sim.run()

The same thing we do with the tool that produces the skymaps, we make a wrapping function to call at our need.

In [27]:
def run_skymap(obs, output, energy=(0.03, 150), roi=5, caldb='prod3b-v2', irf='South_z40_0.5h', wbin=0.02):
    nbin = int(roi*2/wbin)
    skymap = ctools.ctskymap()
    skymap['inobs'] = obs
    skymap['outmap'] = output
    skymap['irf'] = irf 
    skymap['caldb'] = caldb #calibration database, inserito nella fase di configurazione come zip
    skymap['emin'] = energy[0]
    skymap['emax'] = energy[1]
    skymap['usepnt'] = True #use pointing
    skymap['nxpix'] = nbin #Size of the Right Ascension / Galactic longitude axis (in pixels).
    skymap['nypix'] = nbin #Size of the Declination / Galactic latitude axis (in pixels).
    skymap['binsz'] = wbin #Pixel size (in degrees/pixel).
    skymap['bkgsubtract'] = 'IRF' #da testare 'NONE' o 'IRF'
    skymap.execute()
    # if you want to run in memory:
    #sim.run()

Let's fix the pointing.

In [18]:
pointing = (83.6331, 22.5145) #dove si trova l'evento di interesse

## Background simulation and map

We can run the simulation of the background and then use the observation produced to run create the skymap. The simulation will be used in tutorial <code>3_find_prefactor.ipynb</code> to evaluate the background level and find the prefactor required to simulate a source with given significance. Alternatively you can used the template model <code>crab4sigmas.xml</code> in <code>models</code>. The skymap can be used as visual inspection of the simulation and as part of the training of the ML algorithm.

In [28]:
model = '../models/bkg_irf.xml'
run_ctobssim(model=model, pointing=pointing, output='../data/bkg_test_sim.fits')
obs = '../data/bkg_test_sim.fits'
run_skymap(obs=obs, output='../data/bkg_test_sky.fits')

## Source simulation and map

Before this step you should have modified the source model (i.e., crab.xml) with the correct prefactor. You can do so manually or use the provided class in <code>sagsci</code> package for which you can find an example in <code>4_update_source_model.ipynb</code>. For a shortcut I have provided in <code>models</code> folder a template source model with the correct Prefactor for a 4 sigmas simulation of the Crab Nebula. The suggestion is to never go below this value.

In [28]:
model = '../models/crab4sigmas.xml'
run_ctobssim(model=model, pointing=pointing, output='../data/crab_test_sim.fits')
obs = '../data/crab_test_sim.fits'
run_skymap(obs=obs, output='../data/crab_test_sky_3.fits')